<a href="https://colab.research.google.com/github/seriserendipia/HP_QA_system/blob/main/myHarry_Potter_QA_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers faiss-cpu



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
hugging_face_key = userdata.get('Hugging_Face_Token')
deepseek_key = userdata.get('DeepSeek_Token')
from huggingface_hub import login
login(token=hugging_face_key)

In [28]:
import gradio as gr
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np
from openai import OpenAI


# Initialize embedding model
embedder = SentenceTransformer("all-mpnet-base-v2")
prompt = None

# Load and process the Harry Potter text
with open("/content/01 Harry Potter and the Sorcerers Stone.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Split text into chunks
def split_into_chunks(text, max_length=300):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

chunks = split_into_chunks(raw_text)

# Create embeddings for each chunk
chunk_embeddings = embedder.encode(chunks, convert_to_tensor=False)

# Build FAISS index
index = faiss.IndexFlatL2(chunk_embeddings[0].shape[0])
index.add(np.array(chunk_embeddings))

# Initialize Hugging Face QA model
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")


client = OpenAI(api_key=deepseek_key, base_url="https://api.deepseek.com")

# Function to retrieve context based on the question
def get_context(question, top_k=5):
    q_embed = embedder.encode([question])[0]
    _, I = index.search(np.array([q_embed]), top_k)
    context = "\n".join([chunks[i] for i in I[0]])
    return context

# Function to get answer from Hugging Face model
def ask_hf(prompt):
    result = qa_pipeline(prompt, max_new_tokens=100)[0]['generated_text']
    return result.strip()

# Function to get answer from DeepSeek model
def ask_deepseek(prompt):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        stream=False
    )
    return response.choices[0].message.content.strip()

# Combined function to get answers from both models
def ask_both(question):
    context = get_context(question)
    prompt = f"Answer the question based on the context.\nContext: {context}\nQuestion: {question}"
    print("prompt:", prompt)
    import logging
    logging.basicConfig(level=logging.INFO)
    logging.info(prompt)

    hf_answer = ask_hf(prompt)
    ds_answer = ask_deepseek(prompt)
    return hf_answer, ds_answer




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Device set to use cpu


In [29]:
ask_both("How do you make something float in the air?")

prompt: Answer the question based on the context.
Context: the little plastic sticks Muggle children blew bubbles through, except that they were fifty feet high.

Too eager to fly again to wait for Wood, Harry mounted his broomstick and kicked off from the ground. What a feeling — he swooped in and out of the goal posts and then sped up and down the field. 
 Uncle Vernon had hired was still there, with a lot of water in the bottom after the storm.

“How did you get here?” Harry asked, looking around for another boat.

“Flew,” said Hagrid.

“Flew?”

“Yeah — but we’ll go back in this. Not s’pposed ter use magic now I’ve got yeh.”

They settled down in th
Hermione had sunk to the floor in fright; Ron pulled out his own wand — not knowing what he was going to do he heard himself cry the first spell that came into his head: “Wingardium Leviosa!”

The club flew suddenly out of the troll’s hand, rose high, high up into the air, turned slowly over — and d
nix feather, eleven inches, nice and s

('broomstick',
 'To make something float in the air in the context provided, you would use the spell **"Wingardium Leviosa"**. This spell is demonstrated when Ron uses it to levitate the troll\'s club high into the air. \n\nHere’s how it works:\n1. **Cast the spell**: Say "Wingardium Leviosa" clearly and with the proper wand movement.\n2. **Wand movement**: The spell requires a swish and flick motion of the wand (as Hermione instructs earlier in the series).\n3. **Focus**: Concentrate on the object you want to levitate. \n\nExample from the context: \n> Ron pulled out his own wand [...] he heard himself cry the first spell that came into his head: “Wingardium Leviosa!” The club flew suddenly out of the troll’s hand, rose high, high up into the air... \n\nNote: Proper pronunciation and wand technique are crucial—Hermione emphasizes "it\'s Levi-*o*-sa, not Levio-*sa*!"')

In [30]:
# Build Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🧙 Harry Potter Knowledge QA System")

    with gr.Row():
        question_input = gr.Textbox(label="Your Magic Question")
        hf_output = gr.Textbox(label="🧠 Hugging Face Answer")
        ds_output = gr.Textbox(label="🔮 DeepSeek Answer")

    question_input.submit(ask_both, inputs=question_input, outputs=[hf_output, ds_output])

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://660ad5594d755745b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
